In [1]:
import numpy as np
import pandas as pd

pd.__version__


'2.3.1'

In [2]:
file_path = 'course_lead_scoring.csv'

df=pd.read_csv(file_path, na_values=['nan'])
display(len(df))
#First overvuew of data
display(df.head())
#Column list
print(df.columns)
print(" ")

#Types
print("Types")
display(df.dtypes)
print(" ")

#Row size
print("Row size")
print(df.shape[0])
print(" ")

#Index
print("Dataframe index")
display(df.index)
print(" ")

#Clean data
# like series it also has the str method for doing string 
# manipulation what we can do now is to apply the same 
# string function to all column names
df.columns = df.columns.str.lower().str.replace(' ','_')
#display(df.head())

#Column_list
column_list_orig=df.columns.tolist()
print("Null column check")
display(df.isna().any())

1462

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')
 
Types


lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

 
Row size
1462
 
Dataframe index


RangeIndex(start=0, stop=1462, step=1)

 
Null column check


lead_source                  True
industry                     True
number_of_courses_viewed    False
annual_income                True
employment_status            True
location                     True
interaction_count           False
lead_score                  False
converted                   False
dtype: bool

In [3]:
print(df.columns[df.isna().any()].tolist())
print(df.dtypes["annual_income"])
df2=df.copy()
category_list=[]
numerical_list=[]
#Null value replacer
for col in df2.columns[df2.isna().any()].tolist():
    #print(df2.dtypes[col])
    if df2.dtypes[col]=="float64":
        #print(1)
        numerical_list.append(col)
        df2[col]=df2[col].fillna(0.0)
    elif df2.dtypes[col]=="int64":
        df2[col]=df2[col].fillna(0)
        numerical_list.append(col)
    elif df2.dtypes[col]=="object":
        #print(1)
        df2[col]=df2[col].fillna('NA')
        category_list.append(col)
        
#Check replacement has worked        
display(df2.head())
display(df2.isna().any())

#Define list of features
for col in df2.columns.tolist():
    if df2.dtypes[col]=="float64" or df2.dtypes[col]=="int64":
        numerical_list.append(col)
    elif df2.dtypes[col]=="object":
        category_list.append(col)
category_list = list(df2.dtypes[df2.dtypes == 'object'].index)
numerical_list = list(df2.dtypes[(df2.dtypes == 'float64') | (df2.dtypes == 'int64')].index)

for c in category_list:
    df[c] = df[c].str.lower().str.replace(' ', '_')


['lead_source', 'industry', 'annual_income', 'employment_status', 'location']
float64


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


lead_source                 False
industry                    False
number_of_courses_viewed    False
annual_income               False
employment_status           False
location                    False
interaction_count           False
lead_score                  False
converted                   False
dtype: bool

In [4]:
print(df2['industry'].mode())

0    retail
Name: industry, dtype: object


In [5]:
#Print correlation matrix
corr=df2[numerical_list].corr()
display(corr)

display(np.abs(corr.loc['interaction_count', 'lead_score']))
display(np.abs(corr.loc['number_of_courses_viewed', 'lead_score']))
display(np.abs(corr.loc['number_of_courses_viewed', 'interaction_count']))
display(np.abs(corr.loc['annual_income', 'interaction_count']))

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


np.float64(0.009888182496913131)

np.float64(0.004878998354681276)

np.float64(0.023565222882888037)

np.float64(0.02703647240481443)

In [18]:
#Validation framework
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df2, test_size=1/5, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=1/4, random_state=42)

#Check set ratios
print(df_train.shape[0]/df2.shape[0])
print(df_val.shape[0]/df2.shape[0])
print(df_test.shape[0]/df2.shape[0])

#Reset indices
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

#Y sets
y_train_orig = df_train.converted.values
y_val_orig = df_val.converted.values
y_test_orig = df_test.converted.values

y_train = np.log1p(df_train.converted.values)
y_val = np.log1p(df_val.converted.values)
y_test = np.log1p(df_test.converted.values)


df_train0=df_train.copy()
#Delete converted from df
del df_train['converted']
del df_val['converted']
del df_test['converted']

0.5991792065663475
0.20041039671682626
0.20041039671682626


In [19]:
#Mutual information scores
from sklearn.metrics import mutual_info_score
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_train0.converted)

#Mutual information score with categorical variables
mi = df_train0[category_list].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

In [20]:
#Logistic regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

#One-hot encoding
from sklearn.feature_extraction import DictVectorizer

display(numerical_list)
display(category_list)
item_to_remove = 'converted'
numerical_list2= [item for item in numerical_list if item != item_to_remove]

dv = DictVectorizer(sparse=False)

train_dict = df_train[category_list + numerical_list2].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[category_list + numerical_list2].to_dict(orient='records')
X_val = dv.transform(val_dict)

model.fit(X_train, y_train_orig)

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

['lead_source', 'industry', 'employment_status', 'location']

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [48]:
#Model properties
model.intercept_[0]
model.coef_[0].round(3)
y_pred0= model.predict(X_val)
y_pred = model.predict_proba(X_val)[:,1]

#accuracy 0
acc0=(y_val_orig == y_pred0).mean().round(4)
display(acc0)

converted_decision = (y_pred >= 0.5)

#accuracy 1
acc1=(y_val_orig == converted_decision).mean().round(4)
display(acc1)

#accuracy 2
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val_orig
df_pred['correct'] = df_pred.prediction == df_pred.actual
acc2=df_pred.correct.mean().round(4)
display(acc2)

#accuracy 3
from sklearn.metrics import accuracy_score
acc3 = accuracy_score(y_val_orig, y_pred0)
display(np.round(acc3,4))

np.float64(0.6997)

np.float64(0.6997)

np.float64(0.6997)

np.float64(0.6997)

In [40]:
#Remove features to check significance

feature_list=category_list + numerical_list2
acclist=[]
for x in feature_list:
    df_full_train, df_test = train_test_split(df2, test_size=1/5, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size=1/4, random_state=42)


    
    #Reset indices
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    #Remove feature
    del df_train[x]
    del df_val[x]
    del df_test[x]
    
    #Y sets
    y_train_orig = df_train.converted.values
    y_val_orig = df_val.converted.values
    y_test_orig = df_test.converted.values
    
    y_train = np.log1p(df_train.converted.values)
    y_val = np.log1p(df_val.converted.values)
    y_test = np.log1p(df_test.converted.values)
    
    
    df_train0=df_train.copy()
    #Delete converted from df
    del df_train['converted']
    del df_val['converted']
    del df_test['converted']

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

    #One-hot encoding
    
    dv = DictVectorizer(sparse=False)
    items_to_remove = ['converted', x]
    numerical_list3= [item for item in numerical_list if item not in items_to_remove]
    category_list3= [item for item in category_list if item not in items_to_remove]
    
    train_dict = df_train[category_list3 + numerical_list3].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = df_val[category_list3 + numerical_list3].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    model.fit(X_train, y_train_orig)
    y_pred0= model.predict(X_val)
    y_pred = model.predict_proba(X_val)[:,1]


    accx0=(y_val_orig == y_pred0).mean()
    
    converted_decision = (y_pred >= 0.5)
    
    
    #accuracy 3

    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    df_pred['prediction'] = converted_decision.astype(int)
    df_pred['actual'] = y_val_orig
    df_pred['correct'] = df_pred.prediction == df_pred.actual
    accx2=df_pred.correct.mean().round(3)
    acclist.append([x,accx0-acc0])
    

In [41]:
display(acclist)

[['lead_source', np.float64(0.0030716723549488734)],
 ['industry', np.float64(-0.0003412969283276279)],
 ['employment_status', np.float64(-0.003754266211604018)],
 ['location', np.float64(0.009897610921501765)],
 ['number_of_courses_viewed', np.float64(-0.14368600682593857)],
 ['annual_income', np.float64(0.1532423208191127)],
 ['interaction_count', np.float64(-0.14368600682593857)],
 ['lead_score', np.float64(0.006484641638225264)]]

In [44]:
regulist=[0.01, 0.1, 1, 10, 100]
#regulist=[1e-5, 1e-4, 1e-3, 1e-2, 1, 10]
acclistreg=[]

for c in regulist:
    df_full_train, df_test = train_test_split(df2, test_size=1/5, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size=1/4, random_state=42)

   
    #Reset indices
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    
    #Y sets
    y_train_orig = df_train.converted.values
    y_val_orig = df_val.converted.values
    y_test_orig = df_test.converted.values
    
    
    
    df_train0=df_train.copy()
    #Delete converted from df
    del df_train['converted']
    del df_val['converted']
    del df_test['converted']

    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)

    #One-hot encoding
    
    dv = DictVectorizer(sparse=False)
    items_to_remove = ['converted']
    numerical_list3= [item for item in numerical_list if item not in items_to_remove]
    category_list3= [item for item in category_list if item not in items_to_remove]
    
    train_dict = df_train[category_list3 + numerical_list3].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = df_val[category_list3 + numerical_list3].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    model.fit(X_train, y_train_orig)
    y_pred0= model.predict(X_val)
    y_pred = model.predict_proba(X_val)[:,1]
    accx=(y_val_orig == y_pred0).mean()
    converted_decision = (y_pred >= 0.5)
   
    #accuracy 2

    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    df_pred['prediction'] = converted_decision.astype(int)
    df_pred['actual'] = y_val_orig
    df_pred['correct'] = df_pred.prediction == df_pred.actual
    acc2=df_pred.correct.mean()
    acclistreg.append([c,accx])

    print(f"C={c}")
    print("Intercept:", model.intercept_)
    print("Mean |coef|:", np.mean(np.abs(model.coef_)))
    print("Accuracy:", accx)
        

C=0.01
Intercept: [-0.07763057]
Mean |coef|: 0.04647950203402172
Accuracy: 0.6996587030716723
C=0.1
Intercept: [-0.07010903]
Mean |coef|: 0.048882612233649717
Accuracy: 0.6996587030716723
C=1
Intercept: [-0.06914728]
Mean |coef|: 0.049122344576227314
Accuracy: 0.6996587030716723
C=10
Intercept: [-0.06904869]
Mean |coef|: 0.049146287015817
Accuracy: 0.6996587030716723
C=100
Intercept: [-0.06903881]
Mean |coef|: 0.049148680922180533
Accuracy: 0.6996587030716723


In [45]:
display(acclistreg)

[[0.01, np.float64(0.6996587030716723)],
 [0.1, np.float64(0.6996587030716723)],
 [1, np.float64(0.6996587030716723)],
 [10, np.float64(0.6996587030716723)],
 [100, np.float64(0.6996587030716723)]]